In [1]:
import os
import cv2
import torch
import numpy as np
import pandas as pd

import wandb
import datetime
from tqdm.auto import tqdm

from timm.models import resnet18, efficientnetv2_s, vit_tiny_r_s16_p8_224, vit_small_r26_s32_224, InceptionResnetV2, efficientnetv2_m
from torchsummary import summary

In [2]:
from models import ResNet18_recipy

model = ResNet18_recipy().cuda()

model

ResNet18_recipy(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (drop_block): Identity()
        (act1): ReLU(inplace=True)
        (aa): Identity()
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (act2): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding

In [3]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from dataset import *

In [4]:
tf = A.Compose([
    A.Resize(224, 224),
    A.Normalize(),
])

train_dataset = CustomDataset(
            "/opt/ml/level3_cv_finalproject-cv-01/model", is_train=True, tf=tf
        )

In [5]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
            dataset=train_dataset,
            batch_size=4,
            shuffle=True,
            num_workers=8,
            drop_last=True,
        )

In [6]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn.metrics import f1_score, precision_score


cls_criterion = nn.CrossEntropyLoss()
rcp_criterion = nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels, ingredients in train_loader:
        images, labels, ingredients = (
                images.clone().detach().cuda(),
                labels.clone().detach().cuda(),
                ingredients.clone().detach().cuda(),
            )
        print(f"image : {images.shape}")
        print(f"label : {labels.shape}")
        print(f"ingredient : {ingredients.shape}")
        print(f"max : {torch.max(images)}, min : {torch.min(images)}")

        cls_output, rcp_output = model(images)
        print(f"cls_output : {cls_output.shape}")
        print(f"rcp_output : {rcp_output.shape}")
        print(f"max : {torch.max(cls_output)}, min : {torch.min(cls_output)}")
        print(f"max : {torch.max(rcp_output)}, min : {torch.min(rcp_output)}")

        cls_output = F.sigmoid(cls_output)
        rcp_output = F.sigmoid(rcp_output)
        print(f"max : {torch.max(cls_output)}, min : {torch.min(cls_output)}")
        print(f"max : {torch.max(rcp_output)}, min : {torch.min(rcp_output)}")
        print(f"max : {torch.max(labels)}, min : {torch.min(labels)}")
        print(f"max : {torch.max(ingredients)}, min : {torch.min(ingredients)}")

        cls_loss = cls_criterion(cls_output, labels)
        rcp_loss = rcp_criterion(rcp_output, ingredients)
        print(f"Loss : {cls_loss}, {rcp_loss}")

        cls_thr = 0.5
        rcp_thr = 0.5
        cls_output = (cls_output >= cls_thr).float()
        rcp_output = (rcp_output >= cls_thr).float()
        cls_loss = cls_criterion(cls_output, labels)
        rcp_loss = rcp_criterion(rcp_output, ingredients)
        print(f"Loss : {cls_loss}, {rcp_loss}")

        cls_f1 = f1_score(y_pred=cls_output.cpu(), y_true=labels.cpu(), average="macro", zero_division=0)
        rcp_f1 = f1_score(y_pred=rcp_output.cpu(), y_true=ingredients.cpu(), average="macro", zero_division=0)
        cls_precision = precision_score(y_pred=cls_output.cpu(), y_true=labels.cpu(), average="macro", zero_division=0)
        rcp_precision = precision_score(y_pred=rcp_output.cpu(), y_true=ingredients.cpu(), average="macro", zero_division=0)
        print(f"F1-Score : {cls_f1}, {rcp_f1}")
        print(f"Precision Score : {cls_precision}, {rcp_precision}")
        raise
        for i in range(images.shape[0]):
            image = images[i].cpu()
            image = image.permute(1, 2, 0)
            image_np = image.numpy()
            print(train_dataset.classes[labels[i].item()])
            plt.imshow(image_np)
            plt.axis('off')
            plt.show()
        raise

image : torch.Size([4, 3, 224, 224])
label : torch.Size([4, 93])
ingredient : torch.Size([4, 17])
max : 2.6399998664855957, min : -2.1179039478302
cls_output : torch.Size([4, 93])
rcp_output : torch.Size([4, 17])
max : 0.5821062326431274, min : -0.4418811798095703
max : 0.2393122911453247, min : -0.20795363187789917
max : 0.6415519118309021, min : 0.39129284024238586
max : 0.5595442056655884, min : 0.448198139667511
max : 1.0, min : 0.0
max : 1.0, min : 0.0
Loss : 4.534634232521057, 7.097724437713623
Loss : 4.914929270744324, 7.525835633277893
F1-Score : 0.004301075268817204, 0.1156862745098039
Precision Score : 0.002688172043010753, 0.11274509803921567


RuntimeError: No active exception to reraise

In [ ]:
import torch
import matplotlib.pyplot as plt
from torch import cuda
from torch.optim.lr_scheduler import CosineAnnealingLR

In [ ]:
model.to("cuda" if cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
max_epoch = 50
scheduler = CosineAnnealingLR(optimizer, T_max=max_epoch, eta_min=1e-3)

In [ ]:
lrs = []
for epoch in range(max_epoch):
    optimizer.zero_grad()
    optimizer.step()
    scheduler.step()
    current_lr = optimizer.param_groups[0]["lr"]
    lrs.append(current_lr)
fig, ax = plt.subplots()
ax.plot(lrs)
plt.grid()
plt.show()

In [11]:
import json
import os

path = "/opt/ml/level3_cv_finalproject-cv-01/model/data/json/milfeuille"
file_names = os.listdir(path)
for file_name in file_names:
    # new_name = file_name.replace("onirigi", "onigiri")
    with open(os.path.join(path, file_name)) as file:
        data = json.load(file)
    data['food_class'] = "milfeuille"
    with open(os.path.join("/opt/ml/level3_cv_finalproject-cv-01/model/milfeuille", file_name), "w") as file:
        json.dump(data, file)
    